In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

C:\Users\carlo\AppData\Local\Temp\ipykernel_10580\4076755805.py:6: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  opciones.headless=False    # si True, no aperece la ventana (headless=no visible)


In [3]:
perros = pd.read_csv(r'..\dog_breeds.csv',encoding='ISO-8859-1')

In [4]:
perros.head()

,Breed,Country of Origin,Fur Color,Height (in),Color of Eyes,Longevity (yrs),Character Traits,Common Health Problems
0,Labrador Retriever,Canada,"Yellow, Black, Chocolate",21-24,Brown,10-12,"Loyal, friendly, intelligent, energetic, good-...","Hip dysplasia, obesity, ear infections"
1,German Shepherd,Germany,"Black, Tan",22-26,Brown,7-10,"Loyal, intelligent, protective, confident, tra...","Hip dysplasia, elbow dysplasia, pancreatitis"
2,Bulldog,England,"White, Red",12-16,Brown,8-10,"Loyal, calm, gentle, brave","Skin allergies, respiratory issues, obesity"
3,Poodle,France,"White, Black, Brown, Apricot",10-15,"Brown, Blue",12-15,"Intelligent, active, affectionate, hypoallergenic","Hip dysplasia, epilepsy, bladder stones"
4,Beagle,England,"White, Tan, Red, Lemon",13-15,Brown,12-15,"Curious, friendly, energetic, good-natured","Ear infections, hip dysplasia, epilepsy"


In [5]:
perros.columns=[e.lower().replace(' ', '_') for e in perros.columns]

In [6]:
perros = perros.drop('common_health_problems', axis=1)

In [7]:
perros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   breed              117 non-null    object
 1   country_of_origin  117 non-null    object
 2   fur_color          117 non-null    object
 3   height_(in)        117 non-null    object
 4   color_of_eyes      117 non-null    object
 5   longevity_(yrs)    117 non-null    object
 6   character_traits   117 non-null    object
dtypes: object(7)
memory usage: 6.5+ KB


Web Scrapping.


In [8]:
PATH = ChromeDriverManager().install()    # instala driver de chrome

driver = webdriver.Chrome(PATH)      # abre una ventana de chrome

url = 'https://www.thekennelclub.org.uk/search/breeds-a-to-z/'

driver.get(url)

# Para que pinche en el boton de accept de las cookies y salgan todos los registros.
# En caso de no ponerlo, solo aparecen los primeros registros que le da tiempo a cargar antes de que salte el mensaje de cookies.
driver.find_element(By.XPATH, '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]/span').click()


web_perros = driver.find_elements(By.CLASS_NAME,'m-breed-card.m-breed-card--padded')
razas_perros = []

# Iterar por todo el archivo de selenium para que lo convierta en texto.
for dog in web_perros:
    dog = dog.text.strip().split('\n')
    razas_perros.append(dog)

driver.quit()

C:\Users\carlo\AppData\Local\Temp\ipykernel_10580\1799695204.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)      # abre una ventana de chrome


In [9]:
# Paso los datos que tengo en texto a un DataFrame
web_perros = pd.DataFrame(razas_perros)
web_perros.head(1)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,TOY,Affenpinscher,Breed characteristics,SIZE,Small,EXERCISE,Up to 1 hour per day,SIZE OF HOME,Flat/ Apartment,GROOMING,...,SHEDS,Yes,LIFESPAN,Over 12 years,VULNERABLE NATIVE BREED,No,TOWN OR COUNTRY,Either,SIZE OF GARDEN,Small/ medium garden


In [10]:
web_perros.shape

(222, 23)

In [11]:
# Elimino las columnas que no me valen
web_perros = web_perros.drop([0, 2, 3, 5, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 19, 21], axis=1)

In [12]:
# Cambio los nombres de las columnas para que sea mas visual.
web_perros = web_perros.rename(columns={1: 'Breed', 4: 'Size', 6: 'Exercise', 10: 'Grooming', 12: 'Coat_lenght'
                                      , 20: 'Town_country', 22: 'Size_garden'})

In [13]:
# Cambio las columnas a minuscula y cambio los espacios por '_'
web_perros.columns=[e.lower().replace(' ', '_') for e in web_perros.columns]

In [14]:
perros = perros.merge(web_perros, on = 'breed', how='inner') # Inner join para que no me aparezcan nulos.

In [15]:
perros.drop_duplicates(subset='breed', keep='first', inplace=True) # Elimino filas duplicadas, para que no se me repitan las razas

In [16]:
perros.isnull().sum()

breed                0
country_of_origin    0
fur_color            0
height_(in)          0
color_of_eyes        0
longevity_(yrs)      0
character_traits     0
size                 0
exercise             0
grooming             0
coat_lenght          0
town_country         0
size_garden          0
dtype: int64

In [17]:
perros.duplicated().sum()

0

In [18]:
# Limpio columnas para poder meterlo como integer en PowerBI
perros['height_(in)'] = perros['height_(in)'].apply(lambda x: int(x.split('-')[0]))

In [19]:
# Limpio columnas para poder meterlo como integer en PowerBI
perros['longevity_(yrs)'] = perros['longevity_(yrs)'].apply(lambda x: int(x.split('-')[0]))

In [20]:
# Cambio nombre de columnas
perros['coat_lenght'] = perros['coat_lenght'].replace({'Short': 1, 'Long': 3, 'Medium': 2, 'Short & long': 1})

In [21]:
# Cambio nombre de columnas
perros['size_garden'] = perros['size_garden'].replace({'Small/ medium garden': 'medium', 'Large garden': 'large', 'No garden': 'home'})

In [22]:
perros = perros.rename(columns={'country_of_origin': 'country','height_(in)': 'height', 'color_of_eyes': 'eyes','longevity_(yrs)': 'longevity', 'size_garden': 'garden'})

In [23]:
razas_perros = perros['breed'].tolist()

In [24]:
import random

dog_names = pd.read_csv(r'..\dognames.csv',encoding='ISO-8859-1')

dog_names['breed'] = ''

In [25]:
dog_names.shape

(16220, 3)

In [26]:
dog_names = dog_names.sample(n=100)

In [27]:
dog_names

,Row_Labels,Count_AnimalName,breed
14392,TARO,10,
11977,RAINBOW,5,
3814,DEARIE,1,
4843,FENIX,1,
5903,HANNIBAL,3,
...,...,...,...
935,BAODAN,2,
13163,SHALOM,3,
6788,JERZEE,2,
3262,COCOLICIOUS,1,


In [28]:
dog_names = dog_names.drop('Count_AnimalName', axis=1) # Borro la columna que no me hace falta
dog_names['Row_Labels'] = dog_names['Row_Labels'].str.lower().str.capitalize() # Lo pongo todo en minuscula con la primera en mayuscula

In [29]:
dog_names['breed'] = [random.choice(razas_perros) for _ in range(len(dog_names))]

In [30]:
dog_names.head(10)

,Row_Labels,breed
14392,Taro,Basset Hound
11977,Rainbow,Giant Schnauzer
3814,Dearie,Whippet
4843,Fenix,Kerry Blue Terrier
5903,Hannibal,Pekingese
3953,Dez,Beagle
4509,Elidin,Lhasa Apso
3611,Dabbie,Yorkshire Terrier
272,Alice,Yorkshire Terrier
10302,Nelly,Shar Pei


In [31]:
dog_names = dog_names.rename(columns={'Row_Labels': 'name'})

In [32]:
dog_names.shape

(100, 2)

In [33]:
dog_names.isnull().sum()

name     0
breed    0
dtype: int64

In [35]:
perros = perros.merge(dog_names, on = 'breed', how='inner')

In [36]:
perros.shape

(100, 14)

In [37]:
perros.isnull().sum()

breed               0
country             0
fur_color           0
height              0
eyes                0
longevity           0
character_traits    0
size                0
exercise            0
grooming            0
coat_lenght         0
town_country        0
garden              0
name                0
dtype: int64

In [39]:
perros.to_csv(r'..\perros_limpio.csv', index=True)